In [1]:
#final bototrch model

##federated learning with 2 clients

import torch
import math
#import sys
import crypten
#sys.path.append('/content/drive/My Drive')
from utilities import find_ate, find_ate_2, find_statistical_parity_score, find_eqop_score, all_metrics
from load_data_redefined_ate import get_data, load_dataset
from constraint import AverageTreatmentEffectLoss, DemographicParityLoss, EqualOpportunityLoss
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score

#from torch import nn
import crypten.nn as nn
from torch import optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

from torch.utils.data import DataLoader
from torch import nn, optim
from torch.utils.data import TensorDataset
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
from botorch.models import SingleTaskGP, ModelListGP
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch.fit import fit_gpytorch_model
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import qExpectedImprovement
from botorch.optim import optimize_acqf
from itertools import chain
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.acquisition.multi_objective.monte_carlo import (
    qExpectedHypervolumeImprovement,
    qNoisyExpectedHypervolumeImprovement,
)
from botorch.acquisition.multi_objective.objective import IdentityMCMultiOutputObjective
from botorch import fit_gpytorch_mll
import math
from botorch.utils.transforms import unnormalize, normalize
from sklearn.metrics import confusion_matrix
from botorch.utils.multi_objective.hypervolume import Hypervolume
from botorch.utils.multi_objective.pareto import is_non_dominated
from botorch.utils.multi_objective.box_decompositions.non_dominated import (
    FastNondominatedPartitioning,
)
crypten.init()
device = torch.device('cpu')

def create_model(input_dim):
    model = nn.Sequential(
        nn.Linear(input_dim, 64),
        nn.ReLU(),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Linear(32, 1),
        nn.Sigmoid()
    )
    return model

# set parameters
fairness_notion = 'stat_parity' #'ate' 'stat_parity'
num_clients = 3
dataset_name = 'default-age'
epochs = 15 #client training epochs
communication_rounds = 50

if dataset_name == 'adult':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/adult.csv'
    sensitive_feature = 'sex' #'sex': 0 ->female, 1-> male
elif dataset_name == 'adult-age':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/adult.csv'
    sensitive_feature = 'sex' #'sex': 0 ->female, 1-> male
elif dataset_name == 'bank':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/bank-full.csv'
    sensitive_feature = 'marital' #'marital': 0->married, 1-> single
elif dataset_name == 'bank-age':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/bank-full.csv'
    sensitive_feature = 'marital' #'marital': 0->married, 1-> single
elif dataset_name == 'default':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/default.csv'
    sensitive_feature = 'SEX' #'sex': 0 ->female, 1-> male
elif dataset_name == 'default-age':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/default.csv'
    sensitive_feature = 'SEX' #'sex': 0 ->female, 1-> male
elif dataset_name == 'law':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/law.csv'
    sensitive_feature = 'sex' #'sex': 0 ->female, 1-> male
elif dataset_name == 'kdd':
    url = './kdd.csv'
    sensitive_feature = 'sex' #'sex': 0 ->female, 1-> male
elif dataset_name == 'kdd-age':
    url = './kdd.csv'
    sensitive_feature = 'sex' #'sex': 0 ->female, 1-> male
elif dataset_name == 'law-income':
    url = 'https://raw.githubusercontent.com/badarm/Fairness-datasets/main/law.csv'
    sensitive_feature = 'sex' #'sex': 0 ->female, 1-> male
else:
    print("dataset not supported, please update file load_data.py")
    exit()

bal_acc_list = []
fairness_notion_list = []
clients_data,X_test, y_test, sex_list, column_names_list, ytest_potential = load_dataset(url,dataset_name, num_clients, sensitive_feature)
X_test = X_test.to(device)
y_test = y_test.to(device)
global_model = create_model(X_test.shape[1])
global_model = global_model.to(device)

# Define a loss function and optimizer
criterion = nn.BCELoss()

def initialize_model(train_x, train_y):
    models = []
    for i in range(train_y.shape[-1]):
        train_objective = train_y[:,i]
        print("bismillah")
        models.append(
          SingleTaskGP(train_x, train_objective.unsqueeze(-1))
        )
    model = ModelListGP(*models)
    mll = SumMarginalLogLikelihood(model.likelihood, model)
    return model,mll


cost_false_negatives = 10.0 #15 for adult-age
cost_false_positives = 1.0

def calculate_weights(targets, cost_false_negatives=5):
    cost_false_negatives = 5
    # Give higher weight to the positive samples because false negatives cost more
    return torch.where(targets == 1, cost_false_negatives, cost_false_positives)

def evaluate(alpha = 100, lr=0.001, cost_false_negatives=5):
    # Initialize a list to store the parameters of each model
    #params = [torch.zeros_like(param.data) for param in global_model.parameters()]
    params = [crypten.cryptensor(torch.zeros_like(param.data)) for param in global_model.parameters()]
    
    for client_name in clients_data.keys():
        print(client_name)
        X1,y1,s1,y1_potential = get_data(client_name, clients_data)
        X1 = X1.to(device)
        y1 = y1.to(device)
        y1_potential = y1_potential.to(device)
        s1 = s1.to(device)
        model1 = create_model(X1.shape[1])
        model1 = model1.to(device)
        model1.load_state_dict(global_model.state_dict())
        optimizer1 = optim.Adam(model1.parameters(), lr=lr)
        if fairness_notion == 'stat_parity':
            dp_loss = DemographicParityLoss(alpha=alpha)
        elif fairness_notion == 'ate':
            dp_loss = EqualOpportunityLoss(alpha=alpha)
        #dp_loss = AverageTreatmentEffectLoss(alpha=alpha)
        for epoch in range(epochs):
            criterion = nn.BCEWithLogitsLoss(pos_weight=None)
            # Training on Client 1
            optimizer1.zero_grad()
            y_pred = model1(X1)
            #class_weights = find_class_weights(y1)
            #criterion = nn.BCELoss(weight=class_weights)
            weights = calculate_weights(y1,cost_false_negatives)
            criterion = torch.nn.BCEWithLogitsLoss(pos_weight=weights)
            X1_cpu = X1.cpu()
            X1_dataframe = pd.DataFrame(X1_cpu.numpy(), columns=column_names_list)
            y_pred_numpy = y_pred.clone().cpu()
            
            #y1_potential = find_potential_outcomes(X1_dataframe,y_pred_numpy.round().detach().numpy())
            #y1_potential = torch.tensor(y1_potential, dtype=torch.float32)
            fairness_loss = dp_loss(X1, y_pred, s1,y1_potential)
            fairness_loss = fairness_loss.to(device)
            loss = criterion(y_pred.view(-1), y1) + fairness_loss
            loss.backward()
            optimizer1.step()
        print(f'- Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')
        # After training, add the trained model's parameters to the list
        
        for param, param_sum in zip(model1.parameters(), params):
            param_sum.add_(crypten.cryptensor(param.data))
            

    # Compute average of parameters and update global model
    average_params = [param_sum / len(clients_data) for param_sum in params]
    # Copy the averaged parameters into the global model
    with torch.no_grad():
        for param_global, param_avg in zip(global_model.parameters(), average_params):
            param_global.copy_(param_avg.get_plain_text())
    global_model.eval()
    
    # Average the model parameters (weights and biases) and set the averaged parameters to both models
    with torch.no_grad():
        y_pred = global_model(X_test).squeeze()
        y_pred_cls = y_pred.round()
        sensitivity,specificity,bal_acc,G_mean,FN_rate,FP_rate,Precision,f1_sc, acc, auc = all_metrics(y_test.cpu(),y_pred.cpu())
        stat_parity = find_statistical_parity_score(sex_list, y_test,y_pred_cls)
        X_test_cpu = X_test.cpu()
        Xtest_dataframe = pd.DataFrame(X_test_cpu.numpy(), columns=column_names_list)
        y_pred_numpy = y_pred.clone().cpu()
        #ytest_potential = find_potential_outcomes(Xtest_dataframe,y_pred_numpy.round().detach().numpy())
        ate = find_ate_2(y_pred_cls.cpu(), ytest_potential, sex_list)#0 means female-protected attribute
        #acc = (y_pred_cls == y_test).float().mean()
        auprc = average_precision_score(y_test.cpu(), y_pred.cpu())
        print(f'Communication round {round+1}/{communication_rounds}')
        if communication_rounds % 1 == 0:
            print(f'Test accuracy: {acc.item()}')
            print("sensitivity: %s" % sensitivity)
            print("specificity: %s" % specificity)
            print("BalanceACC: %s" % bal_acc)
            print("G_mean: %s" % G_mean)
            print("FN_rate: %s" % FN_rate)
            print("FP_rate: %s" % FP_rate)
            print("Precision: %s" % Precision)
            print("f1_sc: %s" % f1_sc)
            print("statistical parity: %s" % stat_parity)
            print("ate: %s" % ate)
            print("auc: %s" % auc)
            print("auprc: %s" % auprc)
    if fairness_notion == 'stat_parity':
        objectives = torch.tensor([[-stat_parity, bal_acc]]) #the two objectives
    elif fairness_notion == 'ate':
        objectives = torch.tensor([[-ate, bal_acc]]) #the two objectives
    return objectives

def models_have_same_parameters(model1, model2):
    params1 = list(model1.parameters())
    params2 = list(model2.parameters())
    print(params1)
    print("bismillah")
    print(params2)
    if len(params1) != len(params2):
        return False
    
    for p1, p2 in zip(params1, params2):
        if not torch.allclose(p1, p2):
            return False

    return True


# Train the model on both clients

for round in range(communication_rounds):
    
    print(f'Communication round {round+1}/{communication_rounds}')

    bounds = torch.tensor([[100.0,0.0], [2000.0,0.01]])  # bounds on learning rate
    #bounds = torch.tensor([[0.0], [350.0]])
    alpha = torch.tensor([100], dtype=torch.float32)
    alpha = alpha.view(1, -1)

    objectives = evaluate(alpha)
    fairness_notion_list.append(objectives[0,0].item())
    bal_acc_list.append(objectives[0,1].item())
    
    #alpha = normalize(alpha, bounds)
    x_input =  torch.tensor([100,0.001], dtype=torch.float32)#input to the optimization process
    x_input = x_input.view(1, -1)
    #model, mll = initialize_model(alpha, objectives.float())
    model, mll = initialize_model(x_input, objectives.float())

    for i in range(10):  # number of rounds of optimization
        print("Global optimization round:", i)
        fit_gpytorch_mll(mll)
        ref_point=torch.tensor([0.0001, 0.001])
        acq_func = qExpectedHypervolumeImprovement(
            model=model.float(),
            ref_point=torch.tensor([0.001, 0.001]),#problem.ref_point.tolist(),  # use known reference point
            sampler=SobolQMCNormalSampler(sample_shape=torch.Size([128])),
            # define an objective that specifies which outcomes are the objectives
            objective=IdentityMCMultiOutputObjective(outcomes=[0, 1]),
            partitioning = FastNondominatedPartitioning(ref_point, Y = objectives)
            
            # specify that the constraint is on the last outcome
            #constraints=[lambda Z: Z[..., -1]],
        )
        candidate, acq_value = optimize_acqf(
            acq_function=acq_func,
            bounds=bounds,
            q=1,
            num_restarts=300,
            raw_samples=1024,
            options ={"batch_limit": 5, "maxiter": 200}
        )


        #new_candidate = unnormalize(candidate, bounds)
        for i, m in enumerate(model.models):
          # Iterate over models and update the train data for each

            new_objectives = evaluate(candidate[0,0].item(), candidate[0,1].item())#evaluate(candidate.item())

            for i, m in enumerate(model.models):
                train_x = torch.cat([m.train_inputs[0], candidate])
                print("bismillah")
                train_y = torch.cat([m.train_targets, new_objectives[:,i]])
                m.set_train_data(train_x, train_y, strict=False)

    # Now both models have the same parameters and we can continue with the next round of communication


global_model.eval()
    
# Average the model parameters (weights and biases) and set the averaged parameters to both models
with torch.no_grad():
        y_pred = global_model(X_test).squeeze()
        y_pred_cls = y_pred.round()
        sensitivity,specificity,bal_acc,G_mean,FN_rate,FP_rate,Precision,f1_sc, acc, auc = all_metrics(y_test.cpu(),y_pred.cpu())
        stat_parity = find_statistical_parity_score(sex_list, y_test,y_pred_cls)
        X_test_cpu = X_test.cpu()
        Xtest_dataframe = pd.DataFrame(X_test_cpu.numpy(), columns=column_names_list)
        y_pred_numpy = y_pred.clone().cpu()
        #ytest_potential = find_potential_outcomes(Xtest_dataframe,y_pred_numpy.round().detach().numpy())
        ate = find_ate_2(y_pred_cls.cpu(), ytest_potential, sex_list)#0 means female-protected attribute
        #acc = (y_pred_cls == y_test).float().mean()
        auprc = average_precision_score(y_test.cpu(), y_pred.cpu())
        print(f'Test accuracy: {acc.item()}')
        print("sensitivity: %s" % sensitivity)
        print("specificity: %s" % specificity)
        print("BalanceACC: %s" % bal_acc)
        print("G_mean: %s" % G_mean)
        print("FN_rate: %s" % FN_rate)
        print("FP_rate: %s" % FP_rate)
        print("Precision: %s" % Precision)
        print("f1_sc: %s" % f1_sc)
        print("statistical parity: %s" % stat_parity)
        print("ate: %s" % ate)
        print("auc: %s" % auc)
        print("auprc: %s" % auprc)

'''
destination = './results/default/'

if dataset_name == 'adult' or dataset_name == 'bank' or dataset_name == 'law'or dataset_name == 'default'or dataset_name == 'kdd':
    if fairness_notion == 'stat_parity':
        np.save(destination+str(num_clients)+'_bal_acc_stat_parity.npy', np.array(bal_acc_list))
        np.save(destination+str(num_clients)+'_stat_parity.npy', np.array(fairness_notion_list))
    else:
        np.save(destination+str(num_clients)+'_bal_acc_ate.npy', np.array(bal_acc_list))
        np.save(destination+str(num_clients)+'_ate.npy', np.array(fairness_notion_list))
else:
    if fairness_notion == 'stat_parity':
        np.save(destination+str(num_clients)+'_attr_bal_acc_stat_parity.npy', np.array(bal_acc_list))
        np.save(destination+str(num_clients)+'_attr_stat_parity.npy', np.array(fairness_notion_list))
    else:
        np.save(destination+str(num_clients)+'_attr_bal_acc_ate.npy', np.array(bal_acc_list))
        np.save(destination+str(num_clients)+'_attr_ate.npy', np.array(fairness_notion_list))
'''

/Users/muhammadharis/anaconda3/envs/rtrtr/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/muhammadharis/anaconda3/envs/rtrtr/lib/python3.10/site-packages/psmpy/psmpy.py:363: UserWarning: Some values do not have a match. These are dropped for purposes of establishing a matched dataframe, and subsequent calculations and plots (effect size). If you do not wish this to be the case please set drop_unmatched=False
  warnings.warn('Some values do not have a match. These are dropped for purposes of establishing a matched dataframe, and subsequent calculations and plots (effect size). If you do not wish this to be the case please set drop_unmatched=False')
/Users/muhammadharis/anaconda3/envs/rtrtr/lib/python3.10/site-packages/psmpy/psmpy.py:363: UserWarning: Some values do not have a match. These are drop

Communication round 1/50
client_1
- Epoch 15/15, Loss: 1.2560300827026367
client_2
- Epoch 15/15, Loss: 1.2089732885360718
client_3
- Epoch 15/15, Loss: 1.269769310951233
Communication round 1/50
Test accuracy: 0.6754415194935022
sensitivity: 0.3733133433283358
specificity: 0.7617823479005998
BalanceACC: 0.5675478456144678
G_mean: 0.5332762090917637
FN_rate: 0.6266866566716641
FP_rate: 0.23821765209940018
Precision: 0.3093167701863354
f1_sc: 0.3383152173913043
statistical parity: 0.04674745669635791
ate: 0.04637362637362638
auc: 0.6189662238289596
auprc: 0.33389948990928203
bismillah
bismillah


/Users/muhammadharis/anaconda3/envs/rtrtr/lib/python3.10/site-packages/botorch/models/gpytorch.py:129: UserWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  warnings.warn(_get_single_precision_warning(X.dtype), UserWarning)
/Users/muhammadharis/anaconda3/envs/rtrtr/lib/python3.10/site-packages/botorch/models/utils/assorted.py:173: InputDataWarning: Input data is not contained to the unit cube. Please consider min-max scaling the input data.
  warnings.warn(msg, InputDataWarning)
/Users/muhammadharis/anaconda3/envs/rtrtr/lib/python3.10/site-packages/botorch/models/utils/assorted.py:201: InputDataWarning: Input data is not standardized. Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


Global optimization round: 0
client_1
- Epoch 15/15, Loss: 1.194143533706665
client_2
- Epoch 15/15, Loss: 1.157598853111267
client_3
- Epoch 15/15, Loss: 1.2086150646209717
Communication round 1/50
Test accuracy: 0.7410863045651449
sensitivity: 0.4407796101949025
specificity: 0.8269065981148244
BalanceACC: 0.6338431041548634
G_mean: 0.6037247452147751
FN_rate: 0.5592203898050975
FP_rate: 0.17309340188517566
Precision: 0.42120343839541546
f1_sc: 0.43076923076923074
statistical parity: -0.049516935191131733
ate: 0.012087401208740134
auc: 0.6696581015404894
auprc: 0.42159414927808836
bismillah
bismillah
client_1
- Epoch 15/15, Loss: 1.170146107673645
client_2
- Epoch 15/15, Loss: 1.1403008699417114
client_3
- Epoch 15/15, Loss: 1.1996222734451294
Communication round 1/50
Test accuracy: 0.7627457514161946
sensitivity: 0.4542728635682159
specificity: 0.8508997429305912
BalanceACC: 0.6525863032494036
G_mean: 0.6217239442313112
FN_rate: 0.545727136431784
FP_rate: 0.14910025706940874
Precisio

/Users/muhammadharis/anaconda3/envs/rtrtr/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/muhammadharis/anaconda3/envs/rtrtr/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/Users/muhammadharis/anaconda3/envs/rtrtr/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-04 to the diagonal
  warnings.warn(
/Users/muhammadharis/anaconda3/envs/rtrtr/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-03 to the diagonal
  warnings.warn(


...

/Users/muhammadharis/anaconda3/envs/rtrtr/lib/python3.10/site-packages/botorch/optim/initializers.py:403: BadInitialCandidatesWarning: Unable to find non-zero acquisition function values - initial conditions are being selected randomly.
  warnings.warn(


client_1
- Epoch 15/15, Loss: 1.0744792222976685
client_2
- Epoch 15/15, Loss: 1.051312804222107
client_3
- Epoch 15/15, Loss: 1.0931296348571777
Communication round 50/50
Test accuracy: 0.7747417527490836
sensitivity: 0.4827586206896552
specificity: 0.8581833761782348
BalanceACC: 0.670470998433945
G_mean: 0.6436578462060384
FN_rate: 0.5172413793103449
FP_rate: 0.14181662382176521
Precision: 0.49310872894333846
f1_sc: 0.4878787878787879
statistical parity: 0.0156963486257318
ate: 0.03262260127931768
auc: 0.7040339084956236
auprc: 0.4274749629898088
bismillah
bismillah
client_1
- Epoch 15/15, Loss: 1.0743718147277832
client_2
- Epoch 15/15, Loss: 1.0513370037078857
client_3
- Epoch 15/15, Loss: 1.092038631439209
Communication round 50/50
Test accuracy: 0.7740753082305898
sensitivity: 0.4827586206896552
specificity: 0.8573264781491002
BalanceACC: 0.6700425494193777
G_mean: 0.6433364190468152
FN_rate: 0.5172413793103449
FP_rate: 0.14267352185089974
Precision: 0.4916030534351145
f1_sc: 0.4

/Users/muhammadharis/anaconda3/envs/rtrtr/lib/python3.10/site-packages/botorch/optim/initializers.py:403: BadInitialCandidatesWarning: Unable to find non-zero acquisition function values - initial conditions are being selected randomly.
  warnings.warn(


client_1
- Epoch 15/15, Loss: 1.0764528512954712
client_2
- Epoch 15/15, Loss: 1.0525860786437988
client_3
- Epoch 15/15, Loss: 1.0949357748031616
Communication round 50/50
Test accuracy: 0.7764078640453183
sensitivity: 0.48575712143928035
specificity: 0.8594687232219366
BalanceACC: 0.6726129223306084
G_mean: 0.6461370233622133
FN_rate: 0.5142428785607196
FP_rate: 0.1405312767780634
Precision: 0.49693251533742333
f1_sc: 0.49128127369219105
statistical parity: 0.016245497445061846
ate: 0.034631529850746245
auc: 0.7030424376500696
auprc: 0.42817957449263905
bismillah
bismillah
client_1
- Epoch 15/15, Loss: 1.076572060585022
client_2
- Epoch 15/15, Loss: 1.0522366762161255
client_3
- Epoch 15/15, Loss: 1.095177412033081
Communication round 50/50
Test accuracy: 0.7757414195268244
sensitivity: 0.48575712143928035
specificity: 0.8586118251928021
BalanceACC: 0.6721844733160413
G_mean: 0.645814840832403
FN_rate: 0.5142428785607196
FP_rate: 0.14138817480719795
Precision: 0.4954128440366973
f1_s

"\ndestination = './results/default/'\n\nif dataset_name == 'adult' or dataset_name == 'bank' or dataset_name == 'law'or dataset_name == 'default'or dataset_name == 'kdd':\n    if fairness_notion == 'stat_parity':\n        np.save(destination+str(num_clients)+'_bal_acc_stat_parity.npy', np.array(bal_acc_list))\n        np.save(destination+str(num_clients)+'_stat_parity.npy', np.array(fairness_notion_list))\n    else:\n        np.save(destination+str(num_clients)+'_bal_acc_ate.npy', np.array(bal_acc_list))\n        np.save(destination+str(num_clients)+'_ate.npy', np.array(fairness_notion_list))\nelse:\n    if fairness_notion == 'stat_parity':\n        np.save(destination+str(num_clients)+'_attr_bal_acc_stat_parity.npy', np.array(bal_acc_list))\n        np.save(destination+str(num_clients)+'_attr_stat_parity.npy', np.array(fairness_notion_list))\n    else:\n        np.save(destination+str(num_clients)+'_attr_bal_acc_ate.npy', np.array(bal_acc_list))\n        np.save(destination+str(num_c

In [ ]:
Test accuracy: 0.7757414195268244
sensitivity: 0.48575712143928035
specificity: 0.8586118251928021
BalanceACC: 0.6721844733160413
G_mean: 0.645814840832403
FN_rate: 0.5142428785607196
FP_rate: 0.14138817480719795
Precision: 0.4954128440366973
f1_sc: 0.4905374716124149
statistical parity: 0.012353986913504394
ate: 0.03517169211689017
auc: 0.7029448000935264
auprc: 0.42855065334198494

In [3]:
destination = './results/default/'

if dataset_name == 'adult' or dataset_name == 'bank' or dataset_name == 'law'or dataset_name == 'default'or dataset_name == 'kdd':
    if fairness_notion == 'stat_parity':
        np.save(destination+str(num_clients)+'_bal_acc_stat_parity.npy', np.array(bal_acc_list))
        np.save(destination+str(num_clients)+'_stat_parity.npy', np.array(fairness_notion_list))
    else:
        np.save(destination+str(num_clients)+'_bal_acc_ate.npy', np.array(bal_acc_list))
        np.save(destination+str(num_clients)+'_ate.npy', np.array(fairness_notion_list))
else:
    if fairness_notion == 'stat_parity':
        np.save(destination+str(num_clients)+'_attr_bal_acc_stat_parity.npy', np.array(bal_acc_list))
        np.save(destination+str(num_clients)+'_attr_stat_parity.npy', np.array(fairness_notion_list))
    else:
        np.save(destination+str(num_clients)+'_attr_bal_acc_ate.npy', np.array(bal_acc_list))
        np.save(destination+str(num_clients)+'_attr_ate.npy', np.array(fairness_notion_list))